## Demo for SDK (Lite Version)

This example shows how to use SDK to deploy a task using a pre-defined repo source.

- [Getting Started](#Getting-Started)
- [Creating And Deploying Task](#Creating-And-Deploying-Task)
    - [Create task](#create-task)
- [View Deployed Task Results](#View-Deployed-Task-Results)
    - [Follow up task status](#follow-up-task-status-optional)
    - [Show result](#show-result)
- [Update Task](#update-task-optional)
    - [Renew task](#renew-task-optional)
    - [Terminate task](#terminate-task-optional)

# Getting Started


### Initialization

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

wallet_address = os.getenv('WALLET')
private_key = os.getenv('PK')
api_key = os.getenv('API_KEY')

# for testnet dev
orchestrator = swan.resource(
    api_key=api_key, 
    service_name='Orchestrator', 
    login_url='https://swanhub-cali.swanchain.io',      # dev version for testnet login url
    url_endpoint='https://swanhub-cali.swanchain.io'    # dev version for testnet
)

# # for testnet prod
# orchestrator = swan.resource(
#     api_key=api_key, 
#     service_name='Orchestrator'
# )

# # for mainnet
# orchestrator = swan.resource(
#     api_key=api_key, 
#     service_name='Orchestrator',
#     network='mainnet'
# )


## Creating And Deploying Task

Show repository image of pre-defined applications (optional)

In [2]:
repos = orchestrator.get_premade_image()
print(json.dumps(repos, indent=2))

{
  "data": [
    {
      "name": "hello-world",
      "url": "https://github.com/alphaflows/hello.git"
    },
    {
      "name": "tetris",
      "url": "https://github.com/alphaflows/tetris-docker-image"
    }
  ],
  "message": "Getting all example code mappings successfully",
  "status": "success"
}


### Create task

Choose a application repository image name from the above list, and provide wallet address and private key to `create_task` interface to deploy a task.

In [3]:
result = orchestrator.create_task(
    wallet_address=wallet_address,
    app_repo_image='tetris',
    private_key=private_key
)

print(json.dumps(result, indent=2))
task_uuid = result['id']

{
  "data": {
    "config_id": 1,
    "created_at": 1719956467,
    "duration": 3600,
    "ended_at": null,
    "error_code": null,
    "id": 2491,
    "order_type": "Creation",
    "refund_tx_hash": null,
    "region": "global",
    "space_id": null,
    "start_in": 300,
    "started_at": 1719956467,
    "status": "pending_payment_confirm",
    "task_uuid": "12f50986-5a9d-4a25-b8ef-aec362454f94",
    "tx_hash": "0xb43c2943a53cfcfcf4f0696fa907dc1f1549f569eaafaa16b2203e65b4d06f53",
    "updated_at": 1719956479,
    "uuid": "198ffcd1-2a10-4bc7-a45b-488c9fccaa09"
  },
  "message": "Query order status success.",
  "status": "success",
  "tx_hash": "0xb43c2943a53cfcfcf4f0696fa907dc1f1549f569eaafaa16b2203e65b4d06f53",
  "id": "12f50986-5a9d-4a25-b8ef-aec362454f94"
}


If completed steps above, please go to [View Deployed Task Results](#View-Deployed-Task-Results) to view results, or [terminate task](#terminate-task) for more functions related to tasks

## View Deployed Task Results

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [4]:
# Check task info
info = orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [],
    "jobs": [],
    "task": {
      "comments": "all CPs failed",
      "created_at": 1719956467,
      "end_at": 1719960067,
      "id": 4463,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0xaA5812Fb31fAA6C073285acD4cB185dDbeBDC224",
      "source": "v2",
      "start_at": 1719956467,
      "start_in": 300,
      "status": "failed",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719956467,
        "duration": 3600,
        "end_at": 1719960067,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/Qmbs7fcQPmQnJt25Ubv4wpi2hnAckQyV8j5eDFibFWsaRR",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "region": "global",
          "vcpu": "2"
        },
        "start_at": 171995

### Show result

`job_real_uri` is for show the result of application you deployed. Click the url to view application in browser.

In [5]:
result_url = orchestrator.get_real_url(task_uuid)
print(result_url)

[]


## Update Task (optional)

### Renew task (optional)

Extend `task_uuid` by `duration`. Using auto pay automatically makes a transaction to SWAN contract and extends the task.

In [ ]:
renew_task = orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    auto_pay=True, # Optional: Defaults to False, in this demo path set to True
    private_key=private_key
)

if renew_task and renew_task['status'] == 'success':
    print(f"successfully renewed task")

### Terminate task (Optional)

Terminate the task `task_uuid` and get a refund for remaining time

In [ ]:
terminate_status = orchestrator.terminate_task(task_uuid)
if terminate_status['status'] == 'success':
    print(f"Terminated {task_uuid} successfully")
else:
    print(f"Failed to terminate {task_uuid}")